<a href="https://colab.research.google.com/github/AbhianyAnumala/Multimodal-Media-Retrieval-and-Captioning-System/blob/feature_brnach/modality_Accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#WEEK2

## Load embeddings

### Subtask:
Load all the saved image and text embeddings into memory. You might want to organize them in a way that allows for easy lookup, e.g., dictionaries mapping image names to embeddings.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from torch.nn.functional import cosine_similarity
# Check if CUDA is available and set the device accordingly
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")

In [ ]:
import os
import torch

# Define the directories where embeddings are saved
image_embeddings_dir = "/content/drive/MyDrive/AIML_Lab/Capstone_Project/clip_image_embeddings"
text_embeddings_dir = "/content/drive/MyDrive/AIML_Lab/Capstone_Project/clip_text_embeddings"

# Create dictionaries to store embeddings
image_embeddings_dict = {}
text_embeddings_dict = {}

# Load image embeddings
print("Loading image embeddings...")
for filename in os.listdir(image_embeddings_dir):
    if filename.endswith(".pt"):
        image_name = os.path.splitext(filename)[0]
        filepath = os.path.join(image_embeddings_dir, filename)
        try:
            embedding = torch.load(filepath)
            image_embeddings_dict[image_name] = embedding
        except Exception as e:
            print(f"Error loading image embedding {filename}: {e}")

print(f"Loaded {len(image_embeddings_dict)} image embeddings.")

# Load text embeddings
print("Loading text embeddings...")
for filename in os.listdir(text_embeddings_dir):
    if filename.endswith(".pt"):
        image_name = os.path.splitext(filename)[0]
        filepath = os.path.join(text_embeddings_dir, filename)
        try:
            embedding = torch.load(filepath)
            text_embeddings_dict[image_name] = embedding
        except Exception as e:
            print(f"Error loading text embeddings {filename}: {e}")

print(f"Loaded {len(text_embeddings_dict)} text embeddings.")

Loading image embeddings...


<ipython-input-6-bcf4c423fd7f>:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  embedding = torch.load(filepath)
<ipython-input-6-bcf4c423fd7f>:19: FutureWarning: You are u

Loaded 8091 image embeddings.
Loading text embeddings...


<ipython-input-6-bcf4c423fd7f>:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  embedding = torch.load(filepath)
<ipython-input-6-bcf4c423fd7f>:33: FutureWarning: You are u

Loaded 8091 text embeddings.


##**Image-to-text Accuracy**

### Get top N similare embeddings

In [ ]:
def get_top_n_text_results_for_image(image_embeddings, text_embeddings_dict, n=5):
    similarity_scores = {}
    ## compare the given image embedding with all text emdeddings.
    # Assuming image_embeddings is a single embedding for the query image
    # Reshape query embedding for cosine similarity calculation if it's 1D
    if len(image_embeddings.shape) == 1:
        query_embedding_reshaped = image_embeddings.unsqueeze(0) # Use unsqueeze for PyTorch tensors
    else:
        query_embedding_reshaped = image_embeddings


    for image_name, embeddings_list in text_embeddings_dict.items():
        if embeddings_list is not None and len(embeddings_list) > 0: # Added check for empty list
            for index, caption_embedding in enumerate(embeddings_list): # Added enumerate to get index
                if caption_embedding is not None: # Added check for None embedding
                  # Ensure embeddings are on the same device
                  # query_embedding_reshaped is already on 'device' from loading

                  # Reshape caption embedding if it's 1D
                  if len(caption_embedding.shape) == 1:
                      caption_embedding = caption_embedding.unsqueeze(0) # Use unsqueeze for PyTorch tensors


                  # Calculate cosine similarity (assuming it returns a tensor)
                  score = cosine_similarity(query_embedding_reshaped, caption_embedding, dim=1) # Use dim=1 for batch comparison
                  similarity_scores[image_name+"@"+str(index)] = score.item() # Store scalar score and use string index
        else:
            # Handle case where embeddings_list is None or empty for an image
            similarity_scores[image_name] = -1 # Or some other indicator

    ## sort the scores
    sorted_similarity_scores = sorted(similarity_scores.items(), key=lambda item: item[1], reverse=True)
    ## get top n results
    top_n_results = sorted_similarity_scores[:n] # Use the provided n for top results
    return top_n_results

### Comparing embeddings from Search with Actual

In [ ]:
def compare_with_actual_text(top_n_results, image_name):
    for result_key, result_score in top_n_results:
      # Split the key to get the image name and caption index of the result
      result_image_name, result_caption_index_str = result_key.split("@")
      if  result_image_name == image_name:
          return 1
    return 0 # Return 0 if no correct match is found in the top results

### Test for one image embeddings

In [ ]:
## test for one image embedding from dict
image_name_to_get = list(image_embeddings_dict.keys())[0] # Get the first image name as an example
mage_embedding = image_embeddings_dict[image_name_to_get]
top_n_results = get_top_n_text_results_for_image(mage_embedding,text_embeddings_dict,5)
print(top_n_results)
vaa = compare_with_actual_text(top_n_results,image_name_to_get)
print(vaa)

[('395461421_c586b136de@0', 0.3416922390460968), ('2351479551_e8820a1ff3@4', 0.32637324929237366), ('2891162278_fbf96be4f4@0', 0.3214271366596222), ('395461421_c586b136de@3', 0.3180912435054779), ('427167162_2c99779444@1', 0.3175909221172333)]
1


### Test for all embeddings with different top n results



In [ ]:
# Assuming image_embeddings_dict and text_embeddings_dict are loaded
# Assuming get_top_n_text_results_for_image and compare_with_actual_text are defined and corrected

k = 5 # You can set k to the maximum number of top results you want to check for accuracy

top_n_results_dict_image_to_text = {}

# Iterate through each image embedding in the dictionary
for image_name, image_embedding in list(image_embeddings_dict.items())[:1000]:
    # Get the top N text results for the current image embedding
    # Using the corrected get_top_n_text_results_for_image function
    top_n_results_dict_image_to_text[image_name] = get_top_n_text_results_for_image(image_embedding, text_embeddings_dict, k)

accuracy_results = {}

print("Calculating image-to-text accuracy for different top N values...")

# Iterate through different values of N (from 1 to k)
for top_n_accuracy in range(1, k + 1):
    correct_matches_count = 0
    total_images = len(image_embeddings_dict)
    for image_name, top_n_results in top_n_results_dict_image_to_text.items():
        # Check if any of the actual text embeddings for this image are in the top N results
        # Using the corrected compare_with_actual_text function
        # Note: The compare_with_actual_text function returns 1 if a match is found, 0 otherwise.
        is_correct_match_found = compare_with_actual_text(top_n_results, image_name)

        if is_correct_match_found == 1:
            correct_matches_count += 1

    # Calculate accuracy for the current top_n_accuracy value
    accuracy = correct_matches_count / total_images if total_images > 0 else 0
    accuracy_results[top_n_accuracy] = accuracy

    print(f"Accuracy for top {top_n_accuracy} results: {accuracy:.4f}")

# Print all accuracy results
print("\nOverall Accuracy Results:")
for n, acc in accuracy_results.items():
    print(f"Top {n}: {acc:.4f}")






---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---

---


##**Text-to-Image Accuracy**

###get top n image results for text

In [ ]:
def get_top_n_image_results_for_text(text_embedding, image_embeddings_dict, n=5):
    similarity_scores = {}

    # Assuming text_embedding is a single embedding for the query text
    # Reshape query embedding for cosine similarity calculation if it's 1D
    if len(text_embedding.shape) == 1:
        query_embedding_reshaped = text_embedding.unsqueeze(0)
    else:
        query_embedding_reshaped = text_embedding

    for image_name, image_embedding in image_embeddings_dict.items():
        if image_embedding is not None:

            # Reshape image embedding if it's 1D
            if len(image_embedding.shape) == 1:
                image_embedding = image_embedding.unsqueeze(0)
            else:
                image_embedding = image_embedding

            # Calculate cosine similarity
            score = cosine_similarity(query_embedding_reshaped, image_embedding, dim=1)
            similarity_scores[image_name] = score.item() # Store scalar score

    # Sort the scores
    sorted_similarity_scores = sorted(similarity_scores.items(), key=lambda item: item[1], reverse=True)
    # Get top n results
    top_n_results = sorted_similarity_scores[:n]
    return top_n_results

###Compare with actual image

In [ ]:
def compare_with_actual_image(top_n_results, actual_image_name):
    for result_image_name, result_score in top_n_results:
        if result_image_name == actual_image_name:
            return 1  # Found the correct image in the top results
    return 0  # Return 0 if the correct image is not found

### Test for all embeddings with different top n results


In [ ]:
k = 5 # You can set k to the maximum number of top results you want to check for accuracy

top_n_results_dict_text_to_image ={}
evaluated_embeddings_count = 0 # To track the number of text embeddings we actually process
# Iterate through each image and its associated text embeddings
for image_name, text_embeddings_list in list(image_embeddings_dict.items())[:100]:
  if text_embeddings_list is not None and len(text_embeddings_list) > 0:
    # For each text embedding associated with the image
    for i, text_embedding in enumerate(text_embeddings_list):
      if text_embedding is not None:
        # Get the top N image results for the current text embedding
        top_n_results_dict_text_to_image[image_name+"@"+str(i)] = get_top_n_image_results_for_text(text_embedding, image_embeddings_dict, k)
        evaluated_embeddings_count += 1

accuracy_results_text_to_image = {}

print("\nCalculating text-to-image accuracy for different top N values...")

# To evaluate text-to-image, we need to iterate through each text embedding
# and find its corresponding image. The text embeddings are stored per image,
# so we can iterate through the text_embeddings_dict.
total_text_embeddings = sum(len(embeddings) for embeddings in text_embeddings_dict.values() if embeddings is not None)


# Iterate through different values of N (from 1 to k)
for top_n_accuracy in range(1, k + 1):
    correct_matches_count = 0
    for text_name ,top_n_results in top_n_results_dict_text_to_image.items():
        # Split the key to get the image name
        image_name = text_name.split("@")[0]
        # Check if the actual image for this text embedding is in the top N results
        is_correct_match_found = compare_with_actual_image(top_n_results, image_name)
        if is_correct_match_found == 1:
          correct_matches_count += 1

    # Calculate accuracy for the current top_n_accuracy value
    accuracy = correct_matches_count / evaluated_embeddings_count if evaluated_embeddings_count > 0 else 0
    accuracy_results_text_to_image[top_n_accuracy] = accuracy

    print(f"Accuracy for top {top_n_accuracy} results (Text-to-Image): {accuracy:.4f}")

# Print all accuracy results for text-to-image
print("\nOverall Accuracy Results (Text-to-Image):")
for n, acc in accuracy_results_text_to_image.items():
    print(f"Top {n}: {acc:.4f}")

KeyboardInterrupt: 

##**Text-to-Text Accuracy**


###Get top n text results for text

In [ ]:
# To evaluate text-to-text, we compare a given text embedding with all other text embeddings.
# We need to find the most similar text embeddings and check if they belong to the same image
# or are different captions for the same image.

def get_top_n_text_results_for_text(query_text_embedding, text_embeddings_dict, query_image_name, query_caption_index, n=5):
    """
    Finds the top N most similar text embeddings for a given query text embedding,
    excluding the query embedding itself.

    Args:
        query_text_embedding (torch.Tensor): The embedding of the text we are querying with.
        text_embeddings_dict (dict): Dictionary mapping image names to lists of text embeddings.
        query_image_name (str): The name of the image the query text embedding belongs to.
        query_caption_index (int): The index of the query text embedding within its image's list.
        n (int): The number of top results to return.

    Returns:
        list: A list of tuples (result_key, similarity_score) for the top N most similar text embeddings.
    """
    similarity_scores = {}

    # Reshape query embedding for cosine similarity calculation if it's 1D
    if len(query_text_embedding.shape) == 1:
        query_embedding_reshaped = query_text_embedding.unsqueeze(0)
    else:
        query_embedding_reshaped = query_text_embedding

    # Iterate through all text embeddings
    for image_name, text_embeddings_list in text_embeddings_dict.items():
        if text_embeddings_list is not None:
            for index, text_embedding in enumerate(text_embeddings_list):
                if text_embedding is not None:
                    # Create a unique key for each text embedding (image_name@caption_index)
                    current_embedding_key = f"{image_name}@{index}"

                    # Skip the query embedding itself
                    if image_name == query_image_name and index == query_caption_index:
                        continue

                    # Reshape current embedding if it's 1D
                    if len(current_embedding_reshaped.shape) == 1:
                        current_embedding_reshaped = text_embedding.unsqueeze(0)
                    else:
                        current_embedding_reshaped = text_embedding

                    # Calculate cosine similarity
                    score = cosine_similarity(query_embedding_reshaped, current_embedding_reshaped, dim=1)
                    similarity_scores[current_embedding_key] = score.item()

    # Sort the scores
    sorted_similarity_scores = sorted(similarity_scores.items(), key=lambda item: item[1], reverse=True)
    # Get top n results
    top_n_results = sorted_similarity_scores[:n]
    return top_n_results


###Compare with actual text to text

In [ ]:
def compare_with_actual_text_to_text(top_n_results, query_image_name):
    """
    Checks if any of the top N most similar text embeddings belong to the same image
    as the query text embedding.

    Args:
        top_n_results (list): A list of tuples (result_key, similarity_score) for the top N results.
        query_image_name (str): The name of the image the query text embedding belongs to.

    Returns:
        int: 1 if at least one of the top results belongs to the same image, 0 otherwise.
    """
    for result_key, result_score in top_n_results:
        try:
            # Split the key to get the image name
            result_image_name, result_caption_index_str = result_key.split("@")

            # Check if the result image name matches the query image name
            if result_image_name == query_image_name:
                return 1  # Found a text embedding from the same image in the top results
        except ValueError as e:
             # Handle cases where result key format is invalid
             print(f"Error processing result key {result_key}: {e}")
             pass # Skip this result if there's an error

    return 0  # Return 0 if no text embedding from the same image is found in the top results

### Test for all embeddings with different top n results


In [ ]:
k = 5 # You can set k to the maximum number of top results you want to check for accuracy

top_n_results_text_to_text ={}
evaluated_embeddings_count = 0 # To track the number of text embeddings we actually process
# Iterate through each image and its associated text embeddings
for image_name, text_embeddings_list in text_embeddings_dict.items():
  if text_embeddings_list is not None and len(text_embeddings_list) > 0:
    # For each text embedding associated with the image
    for caption_index, text_embedding in enumerate(text_embeddings_list):
      if text_embedding is not None:
        evaluated_embeddings_count += 1
        # Get the top N text results for the current text embedding
        top_n_results_text_to_text[image_name +"@"+str(caption_index)] = get_top_n_text_results_for_text(text_embedding, text_embeddings_dict, image_name, caption_index, top_n_accuracy)


accuracy_results_text_to_text = {}

print("\nCalculating text-to-text accuracy for different top N values...")

# To evaluate text-to-text, we iterate through each individual text embedding
total_text_embeddings = sum(len(embeddings) for embeddings in text_embeddings_dict.values() if embeddings is not None)

# Iterate through different values of N (from 1 to k)
for top_n_accuracy in range(1, k + 1):
    correct_matches_count = 0
    for text_name ,top_n_results in top_n_results_text_to_text.items():
        # Split the key to get the image name
        image_name = text_name.split("@")[0]
        # Check if any of the top N results belong to the same image as the query text
        is_correct_match_found = compare_with_actual_text_to_text(top_n_results, image_name)
        if is_correct_match_found == 1:
          correct_matches_count += 1

    # Calculate accuracy for the current top_n_accuracy value
    accuracy = correct_matches_count / evaluated_embeddings_count if evaluated_embeddings_count > 0 else 0
    accuracy_results_text_to_text[top_n_accuracy] = accuracy

    print(f"Accuracy for top {top_n_accuracy} results (Text-to-Text): {accuracy:.4f}")

# Print all accuracy results for text-to-text
print("\nOverall Accuracy Results (Text-to-Text):")
for n, acc in accuracy_results_text_to_text.items():
    print(f"Top {n}: {acc:.4f}")

In [ ]:
# prompt: get precion matrix for image and text embeding dict

import pandas as pd

# Create a DataFrame to store the accuracy results
# Combine the results from all three evaluation tasks
all_accuracy_results = {
    'Image-to-Text': accuracy_results,
    'Text-to-Image': accuracy_results_text_to_image,
    'Text-to-Text': accuracy_results_text_to_text
}

precision_matrix = pd.DataFrame(all_accuracy_results).T

# Rename columns for clarity (Top 1, Top 2, etc.)
precision_matrix.columns = [f'Top {col}' for col in precision_matrix.columns]

print("\nPrecision Matrix (Accuracy at different Top N values):")
precision_matrix

In [ ]:
import os

directory_path = "/content/drive/MyDrive/AIML_Lab/Capstone_Project/clip_text_embeddings_8k/text_embeddings"

# Check if the directory exists
if os.path.exists(directory_path):
    # List all entries in the directory
    entries = os.listdir(directory_path)

    # Count only files (excluding directories)
    file_count = sum(os.path.isfile(os.path.join(directory_path, entry)) for entry in entries)

    print(f"Number of files in '{directory_path}': {file_count}")
else:
    print(f"Directory not found: '{directory_path}'")

Number of files in '/content/drive/MyDrive/AIML_Lab/Capstone_Project/clip_text_embeddings_8k/text_embeddings': 40455


In [ ]:
# prompt: so we have 3 types of retrieval imagetotext, texttoimage, texttotext. so will the quantitative results, retrieal scores comparision.
# code from start consider u have image and text emdeddings

# Print the precision matrix
precision_matrix

In [ ]:
# prompt: draw a graph
# x axix 1,2,34
# yaxis_1 3.35,1.17,0.673,0.459
# yaxis_2 2.52,2.44,2.54,2.62
# x - label Epoch, y label train_loss, valid_loss

import matplotlib.pyplot as plt
epochs = [1, 2, 3, 4]
train_losses = [4.25, 2.87, 1.673, 0.859]
valid_losses = [2.32, 2.54, 2.84, 3.42]

plt.figure(figsize=(10, 6))
plt.plot(epochs, train_losses, marker='o', label='Train Loss')
plt.plot(epochs, valid_losses, marker='o', label='Valid Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss per Epoch for 30k Dataset')
plt.legend()
plt.grid(True)
plt.show()